In [1]:
import json, sys
import pandas as pd
import numpy as np

'''DATA AND FUNCTIONS DEFINITIONS'''

'''DUMMY DATASETS'''
DUMMY_SUB = '90342.ASDFJWFA' #SUB is OpenID Connect user ID
USER_DIRECTORY = dict()
USER_DIRECTORY['90342.ASDFJWFA'] = '17624554'
TRANSACTIONS_DB = {'id': ['0', '9171354', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'],
                   'userID': ['17624554', '17624554', '17624554', '17624554', '17624554', '17624554', '17624554', '17624554', '17624554', '17624554', '17624554', '17624554', '17624554'],
                   'date': ['2014-08-31', '2014-09-29', '2014-10-29', '2014-11-29', '2014-12-31', '2015-01-31', '2015-02-27', '2015-03-31', '2015-04-29', '2015-05-31', '2015-06-29', '2015-07-31', '2015-07-29'],
                   'time': ['01:25:59', '22:21:51', '22:21:51', '22:21:51', '22:21:51', '22:21:51', '22:21:51', '22:21:51', '22:21:51', '22:21:51', '22:21:51', '22:21:51', '22:21:50'],
                   'amount': ['50.00', '10.00', '5000.00', '5000.00', '50.00', '50.00', '50.00', '50.00', '5000.00', '50.00', '500.00', '500.00', '500.00'],
                   'merchantID': ['999999', '196825339', '196825339', '196825339', '196825339', '196825339', '196825339', '196825339', '196825339', '196825339', '196825339', '196825339', '196825339'],
                   'online': ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
                   'X': ['NaN', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                   'Y': ['NaN', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2']
                  }
tx = pd.DataFrame(data=TRANSACTIONS_DB)
DEMOGRAPHICS_DB = {'userID': ['17624554'], 'gender': ['M'], 'marital': ['single'], 'education': ['university'], 'jobtype': ['quality (public)'], 'income': [2845.0], 'age': ['36'], 'homeLat': ['1'], 'homeLon': ['2'], 'workLat': ['3'], 'workLon': ['4']}
dg = pd.DataFrame(data=DEMOGRAPHICS_DB)

#This function should translate OIDC user ID to database userID
def lookupTransactionsUserID(sub):
    return USER_DIRECTORY[sub]

USERID = lookupTransactionsUserID(DUMMY_SUB)

transactions = tx.copy()
transactions = transactions.drop_duplicates()

#timestamps created out of date and time
transactions['timestamp'] = transactions['date'] + ' ' + transactions['time']
transactions['timestamp'] = pd.to_datetime(transactions['timestamp'])

#spending dataframe has userID, amount spent, timestamp, merchant ID, and extra time data for each transaction
spending = pd.concat([transactions['userID'], transactions['amount'], transactions['timestamp'], transactions['merchantID']], axis=1, keys=['userID', 'amount', 'timestamp', 'merchant'])
spending['weekday'] = spending['timestamp'].dt.weekday_name
spending['day'] = spending['timestamp'].dt.day
spending['month'] = spending['timestamp'].dt.month
spending['days_in_month'] = spending['timestamp'].dt.daysinmonth;

#months is a list that splits spending dataframe into 12 dataframes, each grouped by month the transaction occured
months = [g.reset_index() for n, g in spending.set_index('timestamp').groupby(pd.Grouper(freq='M'))]

#only consider transactions by our user
transactions = transactions[transactions.merchantID == USERID]

#Identifies which transactions are recurring, as those are probably bills
#A recurrent transaction has the same amount and merchant on the same day of the month
#d implies the same day of the month, d3 +- 1 day, d7 +- 3 days, and so on
#returns (dCount, d3Count, d7Count) - number of recurring transactions with a varying
#leeway in day of the month number.
def identifyRecurringTransactions():
  def findSimilarTransactions(pos, row):
    amount = row['amount']
    merchant = row['merchant']
    timestamp = row['timestamp']
    weekday = row['weekday']
    day = row['day']
    maxDays = row['days_in_month']
    
    df = pd.DataFrame({'timestamp' : [], 'userID' : [], 'amount' : [], 'merchant' : [], 'weekday' : [], 'day' : [], 'month' : [], 'days_in_month' : []})
    for i in [x for x in range(12) if x != pos]:
      m = months[i]
      df = df.append(m[(m.merchant == merchant) & (m.amount == amount) & (m.weekday == weekday)])
    row['regulars_weekday'] = df.shape[0]

    df = pd.DataFrame({'timestamp' : [], 'userID' : [], 'amount' : [], 'merchant' : [], 'weekday' : [], 'day' : [], 'month' : [], 'days_in_month' : []})
    for i in [x for x in range(12) if x != pos]:
      m = months[i]
      df = df.append(m[(m.merchant == merchant) & (m.amount == amount) & (m.day == day)])
    row['d'] = df.shape[0]

    #3
    j = 1
    df = pd.DataFrame({'timestamp' : [], 'userID' : [], 'amount' : [], 'merchant' : [], 'weekday' : [], 'day' : [], 'month' : [], 'days_in_month' : []})
    for i in [x for x in range(12) if x != pos]:
      m = months[i]
      m0 = months[i-1]
      if day - j >= 0:
        df = df.append(m[(m.merchant == merchant) & (m.amount == amount) & (m.day >= day - j) & (m.day <= day + j)])
      else:
        df = df.append(m0[(m0.day < m0.days_in_month - (j - day)) & (m0.merchant == merchant) & (m0.amount == amount)])
      if i < 11 and day + j > maxDays:
        m2 = months[i+1]
        df = df.append(m2[(m2.day <= (j + day - maxDays)) & (m2.merchant == merchant) & (m2.amount == amount)])
    row['d3'] = df.shape[0]

    #5
    j = 2
    df = pd.DataFrame({'timestamp' : [], 'userID' : [], 'amount' : [], 'merchant' : [], 'weekday' : [], 'day' : [], 'month' : [], 'days_in_month' : []})
    for i in [x for x in range(12) if x != pos]:
      m = months[i]
      m0 = months[i-1]
      if day - j >= 0:
        df = df.append(m[(m.merchant == merchant) & (m.amount == amount) & (m.day >= day - j) & (m.day <= day + j)])
      else:
        df = df.append(m0[(m0.day < m0.days_in_month - (j - day)) & (m0.merchant == merchant) & (m0.amount == amount)])
      if i < 11 and day + j > maxDays:
        m2 = months[i+1]
        df = df.append(m2[(m2.day <= (j + day - maxDays)) & (m2.merchant == merchant) & (m2.amount == amount)])
    row['d5'] = df.shape[0]

    #7
    j = 3
    df = pd.DataFrame({'timestamp' : [], 'userID' : [], 'amount' : [], 'merchant' : [], 'weekday' : [], 'day' : [], 'month' : [], 'days_in_month' : []})
    for i in [x for x in range(12) if x != pos]:
      m = months[i]
      m0 = months[i-1]
      if day - j >= 0:
        df = df.append(m[(m.merchant == merchant) & (m.amount == amount) & (m.day >= day - j) & (m.day <= day + j)])
      else:
        df = df.append(m0[(m0.day < m0.days_in_month - (j - day)) & (m0.merchant == merchant) & (m0.amount == amount)])
      if i < 11 and day + j > maxDays:
        m2 = months[i+1]
        df = df.append(m2[(m2.day <= (j + day - maxDays)) & (m2.merchant == merchant) & (m2.amount == amount)])
    row['d7'] = df.shape[0]
    
    #all
    df = pd.DataFrame({'timestamp' : [], 'userID' : [], 'amount' : [], 'merchant' : [], 'weekday' : [], 'day' : [], 'month' : [], 'days_in_month' : []})
    for i in [x for x in range(12) if x != pos]:
      m = months[i]
      df = df.append(m[(m.merchant == merchant) & (m.amount == amount)])
    row['all'] = df.shape[0]
    
    return row

  months[0]['regulars_weekday'] = 0
  months[0]['d'] = 0
  months[0]['d3'] = 0
  months[0]['d5'] = 0
  months[0]['d7'] = 0
  months[0]['all'] = 0

  #counts how many times each transaction occurs on a monthly basis with a specific +- amount in day of the month
  recurringTransactions = months.copy()
  for i in range(12):
    recurringTransactions[i] = months[i].apply(lambda x: findSimilarTransactions(i, x), axis=1)
    recurringTransactions[i] = recurringTransactions[i][(recurringTransactions[i].regulars_weekday != 0) | (recurringTransactions[i].d != 0)]
    
  #dCount counts same transactions that happen at the same day of the month, d3Count within +- 1 day of the month
  dCount = 0
  d3Count = 0
  d7Count = 0
  for i in range(12):
    if recurringTransactions[i].shape[0] > 0:
      dCount += recurringTransactions[i]['d'].sum()
      d3Count += recurringTransactions[i]['d3'].sum()
      d7Count += recurringTransactions[i]['d7'].sum()
    
  return (dCount, d3Count, d7Count)


#Monthly spending per userID
#Returns a list of dataframes, one dataframe per month in the dataset
#Each dataframe has sum of total spending by each userID in that month, their
#estimated monthly income from the demographics dataset, a 0/1 flag to see whether their
#monthly spending is greater than their monthly income, and a field with the number
#of credit card transactions the userID made that month
def getMonthlySpending():
    monthsTotalSpending = []
    #months is a list of transactions per month
    for month in months:
        month['amount'] = month['amount'].convert_objects(convert_numeric=True)
        j = month.groupby('userID')['amount'].sum().reset_index()
        j['month'] = month['month']
        monthsTotalSpending.append(j)

    monthsSpendingAndIncome = []
    i = 0
    for month in monthsTotalSpending:
        monthsSpendingAndIncome.append(pd.merge(month, dg, on='userID'))
        monthNow = month['month'].iloc[0]
        monthsSpendingAndIncome[i] = pd.concat([monthsSpendingAndIncome[i]['userID'], monthsSpendingAndIncome[i]['amount'], monthsSpendingAndIncome[i]['income']], axis=1, keys=['userID', 'amount', 'income'])
        monthsSpendingAndIncome[i] = monthsSpendingAndIncome[i].convert_objects(convert_numeric=True)
        dfS = spending.convert_objects(convert_numeric=True)
        monthsSpendingAndIncome[i]['overspending_flag'] = np.where(monthsSpendingAndIncome[i]['amount']>=monthsSpendingAndIncome[i]['income'], 1, 0)
        monthsSpendingAndIncome[i]['n_transactions'] = monthsSpendingAndIncome[i]['userID'].map(dfS['userID'][dfS.month == monthNow].value_counts())
        i+=1
    return monthsSpendingAndIncome


#Main function being called
#Returns (credit_score, total number of recurrent payments, total number of late recurrent payments, getMonthlySpending() result)
#Returns estimeated credit score for the userID supplied as a parameter
#Current formula is credit_score = max(0, 10 - (number of months where spending > income)/2 - (number of late recurring payments))
def getScore(userID):
    monthsTotalSpending = getMonthlySpending()
    mobilityDf = []

    for month in monthsTotalSpending:
        m = month.reset_index()
        m  = m.rename(index=str, columns={"custid": "userID"})
        if len(m) > 0:
            mobilityDf.append(m)
    mobilityDf = pd.concat(mobilityDf)
    
    def countTimesOverspent(userID):
        count = mobilityDf[(mobilityDf.overspending_flag == 1) & (mobilityDf.userID == USERID)].shape[0]
        return count
    
    timesOverspent = countTimesOverspent(userID)
    
    (dCount, d3Count, d7Count) = identifyRecurringTransactions()
    late_payments = int(d7Count - d3Count)
    total_payments = int(dCount/2)

    return (max(0, 10 - int(timesOverspent/2) - late_payments), total_payments, late_payments, monthsTotalSpending)

'''MAIN CODE'''
(score, total_payments, late_payments, monthsTotalSpending) = getScore(USERID)

#Converting each dataframe in monthsTotalSpending list into a JSON object
for i in range(len(monthsTotalSpending)):
    monthsTotalSpending[i] = monthsTotalSpending[i].to_json();

percentile_score = 85

#JSON that is returned
json.dump({"monthly_spending": monthsTotalSpending, "late_payments": late_payments, "total_payments": total_payments, "score": score, "percentile_score": percentile_score}, sys.stdout, indent=4)


/Users/matej/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:167: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/matej/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:178: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/matej/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:179: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/matej/miniconda3/lib/python3.6/site-packages/pandas

{
    "monthly_spending": [
        "{\"userID\":{\"0\":17624554},\"amount\":{\"0\":50.0},\"income\":{\"0\":2845.0},\"overspending_flag\":{\"0\":0},\"n_transactions\":{\"0\":1}}",
        "{\"userID\":{\"0\":17624554},\"amount\":{\"0\":10.0},\"income\":{\"0\":2845.0},\"overspending_flag\":{\"0\":0},\"n_transactions\":{\"0\":1}}",
        "{\"userID\":{\"0\":17624554},\"amount\":{\"0\":5000.0},\"income\":{\"0\":2845.0},\"overspending_flag\":{\"0\":1},\"n_transactions\":{\"0\":1}}",
        "{\"userID\":{\"0\":17624554},\"amount\":{\"0\":5000.0},\"income\":{\"0\":2845.0},\"overspending_flag\":{\"0\":1},\"n_transactions\":{\"0\":1}}",
        "{\"userID\":{\"0\":17624554},\"amount\":{\"0\":50.0},\"income\":{\"0\":2845.0},\"overspending_flag\":{\"0\":0},\"n_transactions\":{\"0\":1}}",
        "{\"userID\":{\"0\":17624554},\"amount\":{\"0\":50.0},\"income\":{\"0\":2845.0},\"overspending_flag\":{\"0\":0},\"n_transactions\":{\"0\":1}}",
        "{\"userID\":{\"0\":17624554},\"amount\":{\"0\":